In [1]:
!pip install audiomentations
!pip install torchsummary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/80.3 kB ? eta -:--:--

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 1.9 MB/s eta 0:00:00


In [2]:
from esc_dataset.esc_dataset import ESCDataset
from esc_utils.esc_utils import train,test,plot_costs,plot_accuracies

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torch.optim import lr_scheduler
from torchsummary import summary
import pandas as pd
import torchvision
import torch

In [4]:
import datetime

In [5]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")

Using device cuda


In [6]:
AUDIO_PATH = "/kaggle/input/esc-50/dataset/audio"
CSV_PATH  = "/kaggle/input/esc-50/dataset/meta/esc50.csv"

In [7]:
train_dataset = ESCDataset(CSV_PATH,AUDIO_PATH,train=True)
test_dataset = ESCDataset(CSV_PATH,AUDIO_PATH,train=False)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [8]:
model = torchvision.models.efficientnet_b0()

In [9]:
for param in model.parameters():
    param.requires_grad = False
model.classifier[0] = nn.Dropout(p=0.4,inplace=True) 
model.classifier[1] = nn.Linear(in_features=1280, out_features=50)

In [10]:
MODEL_PATH = '/kaggle/input/esc-model_v2/pytorch/default/3/esc-model1_2024-08-12_09-35-21.pth'

In [11]:
model.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [12]:
summary(model.to(device),(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              SiLU-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
              SiLU-6         [-1, 32, 112, 112]               0
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             264
              SiLU-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]             288
          Sigmoid-11             [-1, 32, 1, 1]               0
SqueezeExcitation-12         [-1, 32, 112, 112]               0
           Conv2d-13         [-1, 16, 112, 112]             512
      BatchNorm2d-14         [-1, 16, 1

In [13]:
for name,layer in model.named_modules():
    if isinstance(layer,nn.Conv2d):
        if ('features.6'in name) or ('features.7'in name) or ('features.8'in name):
            print(name)
            for param in layer.parameters():
                param.requires_grad = True

features.6.0.block.0.0
features.6.0.block.1.0
features.6.0.block.2.fc1
features.6.0.block.2.fc2
features.6.0.block.3.0
features.6.1.block.0.0
features.6.1.block.1.0
features.6.1.block.2.fc1
features.6.1.block.2.fc2
features.6.1.block.3.0
features.6.2.block.0.0
features.6.2.block.1.0
features.6.2.block.2.fc1
features.6.2.block.2.fc2
features.6.2.block.3.0
features.6.3.block.0.0
features.6.3.block.1.0
features.6.3.block.2.fc1
features.6.3.block.2.fc2
features.6.3.block.3.0
features.7.0.block.0.0
features.7.0.block.1.0
features.7.0.block.2.fc1
features.7.0.block.2.fc2
features.7.0.block.3.0
features.8.0


In [14]:
summary(model.to(device),(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              SiLU-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
              SiLU-6         [-1, 32, 112, 112]               0
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             264
              SiLU-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]             288
          Sigmoid-11             [-1, 32, 1, 1]               0
SqueezeExcitation-12         [-1, 32, 112, 112]               0
           Conv2d-13         [-1, 16, 112, 112]             512
      BatchNorm2d-14         [-1, 16, 1

In [15]:
model = model.to(device)

In [16]:
loss_fn = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(),lr=1e-5,weight_decay=1e-5)
EPOCHS = 30

In [17]:
model_history = train(model,train_loader,loss_fn,optimiser,device,EPOCHS,test_loader)

Epoch 1


|

|

|

|

...

|

|

|

		Cost : nan

	train accuracy : 2.00%

In [ ]:
STORE_PATH = '/kaggle/working'

In [ ]:
current_time = str(datetime.datetime.now())[:-7].replace(' ','_').replace(':','-')
MODEL_PATH = f"{STORE_PATH}/esc-model1_{current_time}.pth"
print(MODEL_PATH)
torch.save(model.state_dict(),MODEL_PATH)

In [ ]:
costs,train_accs,test_accs = model_history

In [ ]:
plot_costs(costs)

In [ ]:
plot_accuracies(train_accs,test_accs)